# Chapter 8: Fractals（フラクタル）

このノートブックは [Nature of Code](https://natureofcode.com/fractals/) の第8章をJuliaで実装したものです。

## 概要

フラクタルとは、**自己相似性**を持つ幾何学的形状です。
部分を拡大すると、全体と同じ（または類似した）構造が現れます。

### フラクタルの種類

1. **決定論的フラクタル**: 完全に自己相似（コッホ曲線、シェルピンスキー三角形）
2. **確率的フラクタル**: 統計的に自己相似（海岸線、雲）

In [ ]:
using Plots

## 8.1 再帰（Recursion）

フラクタルを生成する最も自然な方法は**再帰**です。
関数が自分自身を呼び出すことで、自己相似構造が作られます。

### 再帰の要素

1. **終了条件**: 再帰を止める条件
2. **自己呼び出し**: パラメータを変えて自分自身を呼ぶ
3. **収束**: 各呼び出しで終了条件に近づく

In [ ]:
# シンプルな再帰の例：階乗
function factorial(n)
    if n <= 1
        return 1
    else
        return n * factorial(n - 1)
    end
end

println("5! = $(factorial(5))")

In [ ]:
# 再帰的な円の描画
function recursive_circles!(plt, x, y, r)
    if r < 2
        return
    end
    
    # 円を描画
    θ = range(0, 2π, length=50)
    cx = x .+ r .* cos.(θ)
    cy = y .+ r .* sin.(θ)
    plot!(plt, cx, cy, color=:blue, alpha=0.5, label="")
    
    # 左右に小さな円を再帰的に描画
    recursive_circles!(plt, x + r, y, r * 0.5)
    recursive_circles!(plt, x - r, y, r * 0.5)
end

p = plot(aspect_ratio=:equal, legend=false, axis=false)
recursive_circles!(p, 0, 0, 100)
title!("Recursive Circles")
p

## 8.2 カントール集合（Cantor Set）

最も単純なフラクタルの一つ。線分を3等分し、中央を取り除く操作を繰り返します。

1. 線分を3等分する
2. 中央の1/3を取り除く
3. 残った2つの線分に同じ操作を適用

In [ ]:
function cantor!(plt, x, y, len, depth)
    if depth == 0
        return
    end
    
    # 線分を描画
    plot!(plt, [x, x + len], [y, y], linewidth=3, color=:black, label="")
    
    # 1/3ずつに分割して再帰
    newLen = len / 3
    cantor!(plt, x, y - 20, newLen, depth - 1)           # 左
    cantor!(plt, x + 2 * newLen, y - 20, newLen, depth - 1)  # 右
end

p = plot(xlims=(-10, 410), ylims=(-200, 20), 
         legend=false, axis=false, aspect_ratio=:auto)
cantor!(p, 0, 0, 400, 7)
title!("Cantor Set")
p

## 8.3 コッホ曲線（Koch Curve）

線分を4つの線分に置き換える操作を繰り返すことで作られます。

1. 線分を3等分
2. 中央の1/3を正三角形の2辺で置き換える
3. できた4つの線分に同じ操作を適用

In [ ]:
struct KochLine
    a::Vector{Float64}  # 始点
    b::Vector{Float64}  # 終点
end

function koch_generate(lines::Vector{KochLine})
    newLines = KochLine[]
    
    for line in lines
        a = line.a
        b = line.b
        
        # 線分を5点に分割
        # a----p1----p2----p3----b
        #            /\
        #           /  \
        #         p2    p3
        
        delta = b - a
        
        # 1/3地点
        p1 = a + delta / 3
        
        # 2/3地点
        p3 = a + delta * 2 / 3
        
        # 頂点（60度回転）
        mid = (a + b) / 2
        dx = delta[1] / 3
        dy = delta[2] / 3
        # 60度回転
        angle = -π / 3
        p2 = p1 + [dx * cos(angle) - dy * sin(angle),
                   dx * sin(angle) + dy * cos(angle)]
        
        push!(newLines, KochLine(a, p1))
        push!(newLines, KochLine(p1, p2))
        push!(newLines, KochLine(p2, p3))
        push!(newLines, KochLine(p3, b))
    end
    
    return newLines
end

function draw_koch(generations::Int)
    # 初期の線分
    lines = [KochLine([0.0, 0.0], [400.0, 0.0])]
    
    # 世代を進める
    for _ in 1:generations
        lines = koch_generate(lines)
    end
    
    # 描画
    p = plot(aspect_ratio=:equal, legend=false, axis=false)
    for line in lines
        plot!(p, [line.a[1], line.b[1]], [line.a[2], line.b[2]], 
              color=:blue, linewidth=1)
    end
    title!("Koch Curve (Gen $generations)")
    return p
end

# 複数世代を比較
p1 = draw_koch(0)
p2 = draw_koch(2)
p3 = draw_koch(4)
p4 = draw_koch(6)

plot(p1, p2, p3, p4, layout=(2, 2), size=(800, 400))

## 8.4 コッホ雪片（Koch Snowflake）

正三角形の各辺にコッホ曲線を適用すると、美しい雪片のような形状になります。

In [ ]:
function draw_koch_snowflake(generations::Int)
    # 正三角形の頂点
    size = 300.0
    h = size * √3 / 2
    
    p1 = [0.0, 0.0]
    p2 = [size, 0.0]
    p3 = [size/2, h]
    
    # 3辺（時計回り）
    lines = [
        KochLine(p1, p2),
        KochLine(p2, p3),
        KochLine(p3, p1)
    ]
    
    for _ in 1:generations
        lines = koch_generate(lines)
    end
    
    p = plot(aspect_ratio=:equal, legend=false, axis=false)
    for line in lines
        plot!(p, [line.a[1], line.b[1]], [line.a[2], line.b[2]], 
              color=:blue, linewidth=1)
    end
    title!("Koch Snowflake (Gen $generations)")
    return p
end

draw_koch_snowflake(5)

## 8.5 シェルピンスキー三角形（Sierpiński Triangle）

三角形を4つの小さな三角形に分割し、中央を取り除く操作を繰り返します。

In [ ]:
function sierpinski!(plt, x1, y1, x2, y2, x3, y3, depth)
    if depth == 0
        # 三角形を塗りつぶし
        plot!(plt, [x1, x2, x3, x1], [y1, y2, y3, y1], 
              fill=true, fillcolor=:purple, fillalpha=0.7, 
              linecolor=:black, linewidth=0.5, label="")
        return
    end
    
    # 各辺の中点
    mx1 = (x1 + x2) / 2
    my1 = (y1 + y2) / 2
    mx2 = (x2 + x3) / 2
    my2 = (y2 + y3) / 2
    mx3 = (x3 + x1) / 2
    my3 = (y3 + y1) / 2
    
    # 3つの小三角形を再帰的に描画（中央は除く）
    sierpinski!(plt, x1, y1, mx1, my1, mx3, my3, depth - 1)  # 左下
    sierpinski!(plt, mx1, my1, x2, y2, mx2, my2, depth - 1)  # 上
    sierpinski!(plt, mx3, my3, mx2, my2, x3, y3, depth - 1)  # 右下
end

function draw_sierpinski(depth::Int)
    size = 400.0
    h = size * √3 / 2
    
    p = plot(aspect_ratio=:equal, legend=false, axis=false,
             xlims=(-10, size+10), ylims=(-10, h+10))
    
    sierpinski!(p, 0.0, 0.0, size/2, h, size, 0.0, depth)
    title!("Sierpiński Triangle (Depth $depth)")
    return p
end

p1 = draw_sierpinski(1)
p2 = draw_sierpinski(3)
p3 = draw_sierpinski(5)
p4 = draw_sierpinski(7)

plot(p1, p2, p3, p4, layout=(2, 2), size=(700, 700))

## 8.6 フラクタル・ツリー（Fractal Tree）

木の構造を再帰的に描画します。各枝は一定の比率で短くなり、一定の角度で分岐します。

In [ ]:
function fractal_tree!(plt, x, y, len, angle, depth; 
                       shrink=0.67, branchAngle=π/6)
    if depth == 0 || len < 2
        return
    end
    
    # 枝の終点を計算
    x2 = x + len * sin(angle)
    y2 = y + len * cos(angle)
    
    # 枝を描画
    linewidth = depth * 0.5
    color = depth > 3 ? :brown : :green
    plot!(plt, [x, x2], [y, y2], color=color, linewidth=linewidth, label="")
    
    # 左右に分岐
    newLen = len * shrink
    fractal_tree!(plt, x2, y2, newLen, angle - branchAngle, depth - 1;
                  shrink=shrink, branchAngle=branchAngle)
    fractal_tree!(plt, x2, y2, newLen, angle + branchAngle, depth - 1;
                  shrink=shrink, branchAngle=branchAngle)
end

function draw_tree(depth::Int; branchAngle=π/6)
    p = plot(aspect_ratio=:equal, legend=false, axis=false,
             xlims=(-200, 200), ylims=(0, 350), background_color=:lightblue)
    fractal_tree!(p, 0, 0, 100, 0, depth; branchAngle=branchAngle)
    title!("Fractal Tree (Depth $depth)")
    return p
end

p1 = draw_tree(6, branchAngle=π/8)
p2 = draw_tree(6, branchAngle=π/5)
p3 = draw_tree(8, branchAngle=π/6)
p4 = draw_tree(10, branchAngle=π/7)

plot(p1, p2, p3, p4, layout=(2, 2), size=(800, 800))

## 8.7 確率的フラクタル・ツリー

ランダム性を加えることで、より自然な木の形状を生成できます。

In [ ]:
function stochastic_tree!(plt, x, y, len, angle, depth)
    if depth == 0 || len < 2
        return
    end
    
    # ランダムな変動を加える
    angleVariation = (rand() - 0.5) * 0.2
    lenVariation = 0.8 + rand() * 0.4
    
    x2 = x + len * sin(angle + angleVariation)
    y2 = y + len * cos(angle + angleVariation)
    
    linewidth = depth * 0.5
    color = depth > 3 ? :brown : :green
    plot!(plt, [x, x2], [y, y2], color=color, linewidth=linewidth, label="")
    
    # ランダムな分岐角度と縮小率
    baseAngle = π/6 + (rand() - 0.5) * 0.3
    shrink = 0.6 + rand() * 0.2
    
    newLen = len * shrink
    stochastic_tree!(plt, x2, y2, newLen, angle - baseAngle, depth - 1)
    stochastic_tree!(plt, x2, y2, newLen, angle + baseAngle, depth - 1)
    
    # たまに3本目の枝
    if rand() < 0.3 && depth > 2
        stochastic_tree!(plt, x2, y2, newLen * 0.8, angle, depth - 1)
    end
end

function draw_stochastic_trees()
    plots = []
    for i in 1:4
        p = plot(aspect_ratio=:equal, legend=false, axis=false,
                 xlims=(-200, 200), ylims=(0, 350), background_color=:lightblue)
        stochastic_tree!(p, 0, 0, 80, 0, 8)
        title!("Random Tree $i")
        push!(plots, p)
    end
    plot(plots..., layout=(2, 2), size=(800, 800))
end

draw_stochastic_trees()

## 8.8 L-Systems（リンデンマイヤーシステム）

L-Systemsは、文字列の書き換えルールを使ってフラクタルを生成する形式文法です。

### 構成要素

1. **アルファベット**: 使用可能な文字（F, G, +, -, [, ] など）
2. **公理（Axiom）**: 初期文字列
3. **ルール**: 文字の置換規則

### 描画の解釈

- `F`: 前進して線を描く
- `G`: 前進（線を描かない）
- `+`: 右に回転
- `-`: 左に回転
- `[`: 状態を保存（push）
- `]`: 状態を復元（pop）

In [ ]:
struct LSystem
    axiom::String
    rules::Dict{Char, String}
    angle::Float64
end

function generate(ls::LSystem, generations::Int)
    current = ls.axiom
    
    for _ in 1:generations
        next = ""
        for c in current
            if haskey(ls.rules, c)
                next *= ls.rules[c]
            else
                next *= c
            end
        end
        current = next
    end
    
    return current
end

function draw_lsystem(sentence::String, angle::Float64, len::Float64)
    x, y = 0.0, 0.0
    heading = π/2  # 上向き
    
    stack = Tuple{Float64, Float64, Float64}[]
    lines_x = Vector{Float64}[]
    lines_y = Vector{Float64}[]
    
    for c in sentence
        if c == 'F' || c == 'G'
            x2 = x + len * cos(heading)
            y2 = y + len * sin(heading)
            if c == 'F'
                push!(lines_x, [x, x2])
                push!(lines_y, [y, y2])
            end
            x, y = x2, y2
        elseif c == '+'
            heading += angle
        elseif c == '-'
            heading -= angle
        elseif c == '['
            push!(stack, (x, y, heading))
        elseif c == ']'
            x, y, heading = pop!(stack)
        end
    end
    
    return lines_x, lines_y
end

In [ ]:
# コッホ曲線のL-System
koch_ls = LSystem("F", Dict('F' => "F+F-F-F+F"), π/2)

sentence = generate(koch_ls, 4)
lines_x, lines_y = draw_lsystem(sentence, koch_ls.angle, 3.0)

p = plot(aspect_ratio=:equal, legend=false, axis=false)
for (lx, ly) in zip(lines_x, lines_y)
    plot!(p, lx, ly, color=:blue, linewidth=1)
end
title!("Koch Curve (L-System)")
p

In [ ]:
# シェルピンスキー三角形のL-System
sierpinski_ls = LSystem("F-G-G", 
                        Dict('F' => "F-G+F+G-F", 'G' => "GG"), 
                        2π/3)

sentence = generate(sierpinski_ls, 6)
lines_x, lines_y = draw_lsystem(sentence, sierpinski_ls.angle, 2.0)

p = plot(aspect_ratio=:equal, legend=false, axis=false)
for (lx, ly) in zip(lines_x, lines_y)
    plot!(p, lx, ly, color=:purple, linewidth=0.5)
end
title!("Sierpiński Triangle (L-System)")
p

In [ ]:
# 植物のL-System
plant_ls = LSystem("X", 
                   Dict('X' => "F+[[X]-X]-F[-FX]+X", 'F' => "FF"), 
                   π/7)

sentence = generate(plant_ls, 5)
lines_x, lines_y = draw_lsystem(sentence, plant_ls.angle, 2.0)

p = plot(aspect_ratio=:equal, legend=false, axis=false, 
         background_color=:lightblue)
for (lx, ly) in zip(lines_x, lines_y)
    plot!(p, lx, ly, color=:darkgreen, linewidth=0.5)
end
title!("Plant (L-System)")
p

In [ ]:
# ドラゴンカーブ
dragon_ls = LSystem("FX", 
                    Dict('X' => "X+YF+", 'Y' => "-FX-Y"), 
                    π/2)

sentence = generate(dragon_ls, 12)
lines_x, lines_y = draw_lsystem(sentence, dragon_ls.angle, 2.0)

p = plot(aspect_ratio=:equal, legend=false, axis=false)
for (lx, ly) in zip(lines_x, lines_y)
    plot!(p, lx, ly, color=:red, linewidth=0.3, alpha=0.7)
end
title!("Dragon Curve (L-System)")
p

## 8.9 マンデルブロ集合

最も有名なフラクタル。複素平面上で、漸化式 z_{n+1} = z_n² + c が発散しない点の集合。

In [ ]:
function mandelbrot(c::Complex, maxIter::Int)
    z = 0.0 + 0.0im
    for i in 1:maxIter
        z = z^2 + c
        if abs(z) > 2
            return i
        end
    end
    return maxIter
end

function draw_mandelbrot(xmin, xmax, ymin, ymax, width, height, maxIter)
    img = zeros(height, width)
    
    for i in 1:width
        for j in 1:height
            x = xmin + (i-1) * (xmax - xmin) / (width - 1)
            y = ymin + (j-1) * (ymax - ymin) / (height - 1)
            c = x + y * im
            img[j, i] = mandelbrot(c, maxIter)
        end
    end
    
    return img
end

img = draw_mandelbrot(-2.5, 1.0, -1.25, 1.25, 400, 300, 100)
heatmap(img, c=:inferno, colorbar=false, 
        title="Mandelbrot Set", aspect_ratio=:auto)

In [ ]:
# ズームイン
img_zoom = draw_mandelbrot(-0.75, -0.65, 0.1, 0.2, 400, 400, 200)
heatmap(img_zoom, c=:inferno, colorbar=false, 
        title="Mandelbrot Set (Zoomed)", aspect_ratio=:equal)

## 8.10 ジュリア集合

マンデルブロ集合と関連するフラクタル。固定のcに対して、初期値zが発散しない点の集合。

In [ ]:
function julia(z::Complex, c::Complex, maxIter::Int)
    for i in 1:maxIter
        z = z^2 + c
        if abs(z) > 2
            return i
        end
    end
    return maxIter
end

function draw_julia(c::Complex, xmin, xmax, ymin, ymax, width, height, maxIter)
    img = zeros(height, width)
    
    for i in 1:width
        for j in 1:height
            x = xmin + (i-1) * (xmax - xmin) / (width - 1)
            y = ymin + (j-1) * (ymax - ymin) / (height - 1)
            z = x + y * im
            img[j, i] = julia(z, c, maxIter)
        end
    end
    
    return img
end

# 異なるc値でジュリア集合を描画
c1 = -0.7 + 0.27im
c2 = -0.4 + 0.6im
c3 = 0.285 + 0.01im
c4 = -0.8 + 0.156im

img1 = draw_julia(c1, -1.5, 1.5, -1.5, 1.5, 300, 300, 100)
img2 = draw_julia(c2, -1.5, 1.5, -1.5, 1.5, 300, 300, 100)
img3 = draw_julia(c3, -1.5, 1.5, -1.5, 1.5, 300, 300, 100)
img4 = draw_julia(c4, -1.5, 1.5, -1.5, 1.5, 300, 300, 100)

p1 = heatmap(img1, c=:viridis, colorbar=false, title="c = $c1")
p2 = heatmap(img2, c=:viridis, colorbar=false, title="c = $c2")
p3 = heatmap(img3, c=:viridis, colorbar=false, title="c = $c3")
p4 = heatmap(img4, c=:viridis, colorbar=false, title="c = $c4")

plot(p1, p2, p3, p4, layout=(2, 2), size=(700, 700))

## まとめ

### フラクタルの特徴

1. **自己相似性**: 部分が全体と同じ構造を持つ
2. **無限の複雑さ**: 有限の空間に無限の詳細
3. **非整数次元**: フラクタル次元で特徴づけられる

### 主要なフラクタル

| フラクタル | 生成方法 | 特徴 |
|-----------|----------|------|
| カントール集合 | 中央1/3を除去 | 無限の点、長さ0 |
| コッホ曲線 | 中央を三角形に | 無限の長さ、有限の面積 |
| シェルピンスキー三角形 | 中央の三角形を除去 | 面積0、無限の境界 |
| マンデルブロ集合 | z² + c の収束判定 | 最も有名なフラクタル |

### L-Systemの応用

- 植物のモデリング
- 道路網の生成
- 建築デザイン
- テクスチャ生成